### Test the robustness of SCARP

In [ ]:
import sys
sys.path.append('../Scarp/')
from downstream import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
np.random.seed(1)

random_state = 1
m = 1.5
merge_thre = 3000
beta = 5000

In [ ]:
# data_type = 'blood2k'
# Data_name = ['blood2K_filter30', 'blood2K_filter40',
#              'blood2K_filter50', 'blood2K_filter60',
#              'blood2K_filter70', 'blood2K_filter_var0.5',
#              'blood2K_filter_var0.6', 'blood2K_filter_var0.7',
#              'blood2K_filter_var0.8', 'blood2K_filter_var0.9']

In [ ]:
data_type = 'Sox10KD'
Data_name = ['Sox10KD_filter10', 'Sox10KD_filter20',
             'Sox10KD_filter30', 'Sox10KD_filter40',
             'Sox10KD_filter50', 'Sox10KD_filter_var0.5',
             'Sox10KD_filter_var0.6', 'Sox10KD_filter_var0.7',
             'Sox10KD_filter_var0.8', 'Sox10KD_filter_var0.9']

In [ ]:
louvain_ARI_score = pd.DataFrame(index=Data_name, columns=['score'])
louvain_NMI_score = pd.DataFrame(index=Data_name, columns=['score'])

running_time = np.zeros(len(Data_name))
Peaks_number = np.zeros(len(Data_name))
Kept_component = np.zeros(len(Data_name))

In [ ]:
for d in range(len(Data_name)):
    data_name = Data_name[d]
    print('\n++++++++++++++++++++ %s ++++++++++++++++++++++++' % data_name)

    # ===========================================================================================
    data = sc.read_h5ad('./Processed data/' + data_name + '.h5ad')

    Cells = data.obs.index
    Cells_num, Peaks_num = data.X.shape
    N = Cells_num + Peaks_num
    labels = data.obs['celltype'].astype('category')
    cluster_num = np.unique(labels).shape[0]
    print('Number of Peaks:', Peaks_num)
    print('Number of Cells:', Cells_num)
    print('Number of labels: ', cluster_num)
    Peaks_number[d] = Peaks_num

    # ===========================================================================================
    t, diffusion_mat = SCARP(data=data,
                             m=m,
                             merge_thre=merge_thre,
                             beta=beta,
                             peak_loc=True)
    running_time[d] = t

    if Peaks_num > 50000:
        k = std_plot(data=diffusion_mat,
                     title=data_name,
                     max_k=100,
                     plot_std=False)
    else:
        k = std_plot(data=diffusion_mat,
                     title=data_name,
                     max_k=50,
                     plot_std=False)

    Kept_component[d] = k

    SCARP_score = compute_score(data_name=data_name,
                                diffusion_mat=diffusion_mat,
                                cluster_num=cluster_num,
                                labels=labels,
                                Cells=Cells,
                                kept_comp=k,
                                random_state=1
                                )

    louvain_ARI_score['score'][data_name] = SCARP_score['ARI']['louvain']
    louvain_NMI_score['score'][data_name] = SCARP_score['NMI']['louvain']

## plot

In [ ]:
louvain_ARI_score = louvain_ARI_score.astype('float')
louvain_NMI_score = louvain_NMI_score.astype('float')
running_time = running_time.astype('float')
Peaks_number = Peaks_number.astype('float')

louvain_ARI_score.to_csv('./Results/'+data_type+'_louvain_ARI_score.txt', sep='\t')
louvain_NMI_score.to_csv('./Results/'+data_type+'_louvain_NMI_score.txt', sep='\t')

In [ ]:
if data_type == 'blood2k':
    x = ['30', '40', '50', '60', '70', '', '0.5', '0.6', '0.7', '0.8', '0.9']
elif data_type == 'Sox10KD':
    x = ['10', '20', '30', '40', '50', '', '0.5', '0.6', '0.7', '0.8', '0.9']

y_louvain_ARI = np.append(np.append(np.array(louvain_ARI_score['score'][0:5]), 0),
                          np.array(louvain_ARI_score['score'][5:]))
y_louvain_NMI = np.append(np.append(np.array(louvain_NMI_score['score'][0:5]), 0),
                          np.array(louvain_NMI_score['score'][5:]))
running_time1 = np.append(np.append(np.array(running_time[0:5]), 0),
                          np.array(running_time[5:]))
Peaks_number1 = np.append(np.append(np.array(Peaks_number[0:5]), 0),
                          np.array(Peaks_number[5:]))

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(6, 3.5))
plt.bar(x=x, height=y_louvain_ARI,
        label='louvain_ARI',
        color=sns.cubehelix_palette(8, start=.5, rot=-.75)[3])
plt.bar(x=x, height=y_louvain_NMI,
        bottom=y_louvain_ARI,
        label='louvain_NMI',
        color=sns.cubehelix_palette(8, start=.5, rot=-.75)[5])
plt.legend(bbox_to_anchor=(0.35, 1.01), loc=0, borderaxespad=0)
plt.ylabel('score')
plt.subplots_adjust(right=0.77)
plt.savefig('./Results/' + data_type + '_Score.svg', bbox_inches='tight')
plt.show()

In [ ]:
if data_type == 'blood2k':
    s = 2*running_time1
    y_lim = 500
elif data_type == 'Sox10KD':
    s = 6*running_time1
    y_lim = 100

fig1, ax = plt.subplots(figsize=(5.5, 2.5))
sns.scatterplot(x=x, y=running_time1,
                palette='rocket', s=s,
                hue=running_time1, ax=ax)
plt.ylabel('Running time (s)')
plt.ylim([0, y_lim])
ax.get_legend().set_visible(False)
plt.savefig('./Results/' + data_type + '_time.svg', bbox_inches='tight')
plt.show()

In [ ]:
if data_type == 'blood2k':
    y_lim = 120000
elif data_type == 'Sox10KD':
    y_lim = 60000
    
fig2, ax = plt.subplots(figsize=(5.5, 2.5))
sns.scatterplot(x=x, y=Peaks_number1,
                palette='rocket', s=0.9*Peaks_number1/100,
                hue=Peaks_number1, ax=ax)
plt.ylabel('Peaks number')
plt.ylim([0, y_lim])
plt.subplots_adjust(left=0.15)
ax.get_legend().set_visible(False)
plt.savefig('./Results/' + data_type + '_peaksnum.svg', bbox_inches='tight')
plt.show()